<a href="https://colab.research.google.com/github/teshiebronk/teshieColab/blob/main/assignment_6_ENGS_108_Fall_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ENGS 108 Fall 2021 Assignment 6**

*Due October 23 2021

**Instructors:** George Cybenko

**TAs:** Clement Nyanhongo, Jack Sadoff

**Student** Teshie Bronk
---

## **Rules and Requirements**


1.   You are only allowed to use Python packages that are explicity imported in 
the assignment notebook or are standard (bultin) python libraries like random, os, sys, etc, (Standard Bultin Python libraries will have a Python.org documentation). For this assignment you may use:
  *   [numpy](https://numpy.org/doc/stable/)
  *   [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html)
  *   [scikit-learn](https://scikit-learn.org/stable/)
  *   [matplotlib](https://matplotlib.org/)
  *   [tensorflow](https://www.tensorflow.org/)

2.   All code must be fit into the designated code or text blocks in the assignment notebook. They are indentified by a **TODO** qualifier.

3. For analytical questions that don't require code, type your answer cleanly in Markdown. For help, see the [Google Colab Markdown Guide](https://colab.research.google.com/notebooks/markdown_guide.ipynb).

---

In [354]:
''' Import Statements '''
from collections import defaultdict
import copy
import itertools
import numpy as np
import time

import random
# Don't mess with this (gives reproducible results)
random.seed(444)

## **Problem 1: Reinforcement Learning**
In this problem we will play a game of cops and robbers. The game is played on a fixed undirected, simple, and finite graph $G$. There are two players, a cop and a robber. It is the goal of the cop to catch the robber in as few moves as possible. 

The graph $G$ has the following properties:
  - A total of $m$ nodes.
  - It contains a single $n$ node cycle, where $n\leq m$, and random additional edges to make the graph connected.

The game starts, with the cop taking their choice of vertex in $G$ and then the robber selects a random vertex in $G$ that is not occupied by the cop. At every point in the game both players know the positions of each other, and in this version of the problem we will say that the robber is drunk (i.e. they will randomly choose there next that instead of employing a policy).

The availabe actions of the cop and associated reward function is:
  - Move to a node not connected to their present node (and the cop stays in the current position): -5.
  - Move to an adjacent node (including staying at current node): -1.
  - Move to the node occupied by robber: +100.
>
> **Part 1** Building a Graph Class.
>> **(a)** Using the provided skeleton build a general graph class for this problem for $n$ nodes. You are expected to implement *add_edge*, *make_random_graph*, *check_connected*.

In [355]:
class Graph:
  """ Our graph class.
  Args:
    n: Number of nodes in our cycle.
    m: Total number of nodes in graph, where n >= m.
  """
  def __init__(self, n, m):
    self.n = n
    self.m = m
    # A default dict is just a dict that won't raise a KeyError, it instead fillsds
    # the unknown key with a default, in our case an empty list.
    self.G = defaultdict(list) #dictionary just links a key to some value
    #difference between this and a regular dictionary 
    #key is node and this is linked to list of everything its attached

    
  def add_edge(self, u, v):
    """
    Make a function that will add an edge to the graph.
    """
    self.G[u].append(v)
    self.G[v].append(u)


  def make_random_graph(self):
    """ First make a cycle of given length and then add random additional edges
    in such a way that the final graph will be connected.
    """
    #TODO: Make n length cycle first
    print("here")
    for i in range((self.n)-1):
      if i == self.n -1:
        print("m")
        continue #break out of loop
      self.add_edge(i,i+1)
    self.add_edge(0,self.n-1) #ie. make the final to first connection


    #TODO: Add additional nodes until random graph is connected
    total_ext = self.m
    counter = self.n
    while counter < total_ext:
      _graph = copy.copy(self)
      #TODO: Use the graph copy to add random nodes/edges
      node_to_add_on = random.randint(0, self.n -1)
      if len(self.G[node_to_add_on]) != 2:
        temp_node = self.G[node_to_add_on][-1]
        self.add_edge(temp_node,counter)
      else:
        self.add_edge(node_to_add_on,counter)
      counter = counter + 1

    if self.check_connected(_graph.G):
      # If it is set the current graph's adjacency matrix equal to the copy's.
      print('true')
      self.graph = _graph.G
      # Return (i.e. break the loop)
      return True
    else:
      return False

  def check_connected(self, G):
    """ Perform a Depth First Search.
    """
    start_node = random.choice(list(G.keys()))
    return dfs(G, start_node, self.m)

def dfs(G,u,m):
  """ Implement your own depth first search. Many online resources for this.
  Args:
    G: Is the graph (adjency matrix) we want to test.
    visited: Is a dictionary that keeps track of the nodes you've visited.
    u: a starting node.
    parent: The parent node for DFS so that you can cancel recursion if you complete the cycle.
  Returns:
    True: If all nodes have been visited at least once.
    False: Otherwise.
  """

  #algorithm that follows a path and marks nodes that it's visited and ensures that its visited every node on the graph 
  #TODO: Implement. Hint: Make sure to keep track of the parent node for a DFS search,
  # as a cycle leads to an infinite recursion, so you have to make sure you break the cycle.
  stack = [] #creates list stack
  stack.append(u)
  visited = set()
  while stack:
    curr = stack.pop()#pulling the last one added on the stack and deleting it    
    visited.add(curr) 
    if len(visited) == m:
      return True
    all_neighbors = G[curr]  #neighbros of G[i]
    for neighbor in all_neighbors:
      if neighbor not in visited:
        stack.append(neighbor)
  
  return False 



>> **(b)** Test out your graph class with a cops and robbers graph of $n=5$ and $m=10$.

In [357]:
#TODO: Your code goes here.
g = Graph(5,10)
bl = g.make_random_graph()
print(g.G)
#Great this appears to work. This prints each node with its associated connected nodes. 

here
true
defaultdict(<class 'list'>, {0: [1, 4], 1: [0, 2], 2: [1, 3, 5], 3: [2, 4, 7], 4: [3, 0, 6], 5: [2, 9], 6: [4, 8], 7: [3], 8: [6], 9: [5]})


> **Part 2** Understanding the state space, i.e. the Game.
>> **(a)** Given the graph class you've created in Part 1. Develop a Cops and Robbers game class. Use the skeleton below to implement the following functions first: *get_successors*, *terminal_test*, *result*. 


In [358]:
class CopsAndRobbers:
  def __init__(self, graph, start_state, rewards_table=None):
    """ This is the cops and robbers game class.
    Args:
      start_state: The starting state of the cop position and robber position in
        the graph. Should be a tuple of form (cop_pos, rob_pos).
      G: The graph adjacency matrix
      state: current state in the game.
      reward_table: The (state, actions) reward dictionary that you will eventually implement.
    """
    self.graph = graph
    self.G = graph.G
    self.state = start_state
    self.rewards_table = rewards_table
  
  
  def terminal_test(self, state):
    #TODO: Implement.
    if self.state[0] == self.state[1]: 
      return True
    return False 

  def get_successors(self, state):
    """ Return a list of successor states that can be reached from the current state.
    Hint: Remember only the cop can choose their action.
    """
    #TODO: Implement.
    return (self.G[state[0]])
    #self.G[state[1]]

  def result_rob(self, state):
    """ This function should return the state after the cop has made their move,
    and the drunk robber has moved accordingly.
    Args:
      state: Current state of (cop, rob).
      next_state: The state after the cop has moved (next_cop, rob). Calculated from get_successors.
    """
    # TODO: Implement.
    next_state_rob = random.choice(self.G[state[1]]) # and this is decided with the reward table
    state_cop = self.state[0]
    self.state = (state_cop,next_state_rob)
    #return (next_state_cop, next_state_rob)

  def result_cop(self, state, next_state):
    next_state_cop = next_state# and this is decided with the reward table
    state_rob = self.state[1]
    self.state = (next_state_cop,state_rob)
    #return (next_state_cop, next_state_rob)

#The availabe actions of the cop and associated reward function is:
#Move to a node not connected to their present node (and the cop stays in the current position): -5.
#Move to an adjacent node (including staying at current node): -1.
#Move to the node occupied by robber: +100.
  def utility(self, state, action):
    return self.rewards_table[(state, action)]

In [359]:
c = CopsAndRobbers(g,(0,4),None)
print(c.get_successors((0,2)))

[1, 4]


>> **(b)** In reinforcement learning we are often interested in calculating a rewards table that has possible states as its rows and possible actions as its columns and filled in with the associated reward given the Q(state, action) pair. Calculate the rewards table for any given graph. *Hint: This should be an $m^2$ x $m$ matrix or a dictionary with $m^3$ keys such that the keys are (state, action) tuples.*  

In [360]:
def calculateRewardsTable(graph):
  """ Make a rewards table dictionary of the from table[(state, action)] = reward.
  """
  #TODO: implement
  #states
  table = defaultdict(list)

  for cop in range(graph.m):
    for rob in range(graph.m):
      for action in range(graph.m):
        if (action in graph.G[cop]) & (action == rob):
          table[(cop,rob),action] = 100
        elif (action in graph.G[cop]) & (action != rob):
          table[(cop,rob),action] = -1
        else: 
          table[(cop,rob),action] = -5
  return table

In [361]:
aa = calculateRewardsTable(g)
len(aa)

1000

>> **(c)** Now that we have our reward table, try to solve the problem in a brute-force manner (without reinforcement learning). I.e. try to reach the terminal state, or find get a reward of 100. 

In [362]:
epochs = 0
rewards = []
penalties = 0
#TODO: Instantiate your graph
g = Graph(5,10)
g.make_random_graph()

#TODO: Build your rewards table
Rewards = calculateRewardsTable(g)

#TODO: Instantiate your game.
#arbitrary start state
start_state = (2,7)
game = CopsAndRobbers(g,start_state,Rewards)

# Simulation loop
playing = True
penalty = 0
while playing:
  #TODO: Get the next state, check for terminal state, and get reward
  next_state_cop = random.randint(0, g.m - 1)
  if next_state_cop in game.G[game.state[0]]: #is our next_state for the cop valid from our current one
    penalty = penalty + game.utility(game.state,next_state_cop) #add penalty from the rewards table
    game.result_cop(game.state,next_state_cop)
    playing = not game.terminal_test(game.state)
    if playing == False:
      continue
      print("playing")
    game.result_rob(game.state)
    playing = not game.terminal_test(game.state) #this checks if robber moved 
    if playing == False:
      penalty == penalty + 100 # adds points because the robber landed on the cop 
      print("playing")
      continue
    
  else:
    penalty = penalty + game.utility(game.state,next_state_cop)
    game.result_rob(game.state)
    playing = not game.terminal_test(game.state)
    if playing == False:
      penalty = penalty+100

  # Make sure to break if you reach the terminal state.

here
true


In [363]:
print(game.state)
print(penalty)


(2, 2)
85


As you can see the game state reached the terminal state where the cop was on the robber and here is the associated penalty of that game.

> **Part 3** Q-learning. 
>
> Up to this point we have build a graph class, built a game class, ran a brute-force simulation of an agent traversing the space randomly, and now we will dive into Q-learning in the hopes of maximizing the rewards and efficiency of capturing the drunk robber. 
>
>> **(a)** Using the skeleton from the brute force method, implement a training loop to learn a Q-table for a given graph and game. 

In [391]:
#new rewards table
n = 5
m = 10

q_graph = Graph(n, m)
q_graph.make_random_graph()

#define the shape of the environment (i.e., its states)
environment_rows = m
environment_columns = m

#Create a 3D numpy array to hold the current Q-values for each state and action pair: Q(s, a) 
#The array contains 11 rows and 11 columns (to match the shape of the environment), as well as a third "action" dimension.
#The "action" dimension consists of 4 layers that will allow us to keep track of the Q-values for each possible action in
#each state (see next cell for a description of possible actions). 
#The value of each (state, action) pair is initialized to 0.
q_values = np.zeros((environment_rows, environment_columns, m))
#Create a 2D numpy array to hold the rewards for each state. 
#The array contains 11 rows and 11 columns (to match the shape of the environment), and each value is initialized to -1.
rewards = np.full((m,m,m), -5.) #defaults to -5

for cop in range(m):
  for rob in range(m):
    for action in range(m):
      if (action in q_graph.G[cop]) & (action == rob):
        rewards[cop,rob,action] = 100
      elif (action in q_graph.G[cop]) & (action != rob):
        rewards[cop,rob,action] = -1


here
true


In [392]:
#define a function that determines if the specified location is a terminal state
def is_terminal_state(current_row_index, current_column_index):
  #if the reward for this location is -1, then it is not a terminal state (i.e., it is a 'white square')
  if current_row_index == current_column_index:
    return True
  else:
    return False

In [393]:
#define a function that will choose a random, non-terminal starting location
def get_starting_location():
  #get a random row and column index
  current_row_index = np.random.randint(environment_rows)
  current_column_index = np.random.randint(environment_columns)
  #continue choosing random row and column indexes until a non-terminal state is identified
  #(i.e., until the chosen state is a 'white square').
  while is_terminal_state(current_row_index, current_column_index):
    current_row_index = np.random.randint(environment_rows)
    current_column_index = np.random.randint(environment_columns)
  return current_row_index, current_column_index

In [394]:
#define an epsilon greedy algorithm that will choose which action to take next (i.e., where to move next)
def get_next_action(current_row_index, current_column_index, epsilon):
  #if a randomly chosen value between 0 and 1 is less than epsilon, 
  #then choose the most promising value from the Q-table for this state.
  if np.random.random() < epsilon:
    return np.argmax(q_values[current_row_index, current_column_index])
  else: #choose a random action
    return np.random.randint(m)

In [395]:
#define a function that will get the next location based on the chosen action
def get_next_location(current_row_index, current_column_index, action_index):
  new_row_index = current_row_index
  new_column_index = current_column_index 

  if action_index in q_graph.G[current_row_index]:
    new_row_index = action_index
  if new_row_index != new_column_index: # only move if not terminal state
    new_column_index = random.choice(q_graph.G[current_column_index]) 

  return new_row_index, new_column_index

In [396]:
#Define a function that will get the shortest path between any location within the warehouse that 
#the robot is allowed to travel and the item packaging location.
def get_shortest_path(start_row_index, start_column_index):
  #return immediately if this is an invalid starting location
  if is_terminal_state(start_row_index, start_column_index):
    return []
  else: #if this is a 'legal' starting location
    current_row_index, current_column_index = start_row_index, start_column_index
    shortest_path = []
    shortest_path.append([current_row_index, current_column_index])
    #continue moving along the path until we reach the goal (i.e., the item packaging location)
    while not is_terminal_state(current_row_index, current_column_index):
      #get the best action to take
      action_index = get_next_action(current_row_index, current_column_index, 1.)
      #move to the next location on the path, and add the new location to the list
      current_row_index, current_column_index = get_next_location(current_row_index, current_column_index, action_index)
      shortest_path.append([current_row_index, current_column_index])
    return shortest_path

In [397]:
#define training parameters
epsilon = 0.9 #the percentage of time when we should take the best action (instead of a random action)
discount_factor = 0.9 #discount factor for future rewards
learning_rate = 0.9 #the rate at which the AI agent should learn

#run through 100 training episodes and track the following factors
reward_tot = 0
reward_tracker =0
steps = 0
steps_tot = 0
time_ave = 0
t1 = 0
for episode in range(100):
  #get the starting location for this episode
  row_index, column_index = get_starting_location() #gives a cop (row) and robber (col) loc
  reward_tot = reward_tracker + reward_tot
  steps_tot = steps_tot + steps
  reward_tracker = 0 
  steps = 0
  time_ave = t1 + time_ave


  #continue taking actions (i.e., moving) until we reach a terminal state
  #(i.e., until we reach the item packaging area or crash into an item storage location)
  t0= time.process_time()
  while not is_terminal_state(row_index, column_index):
    #choose which action to take (i.e., where to move next)
    action_index = get_next_action(row_index, column_index, epsilon)

    #perform the chosen action, and transition to the next state (i.e., move to the next location)
    old_row_index, old_column_index = row_index, column_index #store the old row and column indexes
    row_index, column_index = get_next_location(row_index, column_index, action_index)
    
    #receive the reward for moving to the new state, and calculate the temporal difference
    reward = rewards[old_row_index, old_column_index,action_index] #assign associated reward from the most recent action
    reward_tracker = reward_tracker + reward
    old_q_value = q_values[old_row_index, old_column_index, action_index]
    temporal_difference = reward + (discount_factor * np.max(q_values[row_index, column_index])) - old_q_value

    #update the Q-value for the previous state and action pair
    new_q_value = old_q_value + (learning_rate * temporal_difference)
    q_values[old_row_index, old_column_index, action_index] = new_q_value

    steps = steps+1
  t1 = time.process_time() - t0

#average reward across 100 epochs
reward_tot = reward_tot/100
steps_tot = steps_tot/100
time_ave = time_ave/100

>> **(b)** Evalute your new Q-learning agent over a 100 epochs, by choosing your actions based on the argmax of the Q-table caluculated in (a) and report the average number of penalities, average time, and average number of steps it took to find the robber with your new Q-learning strategy. 

In [398]:
print('Average Reward Per Epoch: ' +str(reward_tot))
print('Average Steps Per Epoch: ' + str(steps_tot))
print('Average Time Per Epoch: ' + str(time_ave)) 

Average Reward Per Epoch: 48.52
Average Steps Per Epoch: 6.39
Average Time Per Epoch: 9.786122000093655e-05


Average Reward Per Epoch: 48.52

Average Steps Per Epoch: 6.39

Average Time Per Epoch: 9.786122000093655e-05

>> **(c)** Compare your results with the brute-force method used in Part 2 and comment on the improvement. For instance, try varying graph configurations and look for any signs of improvement in certian instances. 

In [403]:
#Create a test graph to apply to both the brute force and other method
n = 5
m = 10
g = Graph(n,m)
g.make_random_graph()
#Run on Brute Force Method
Rewards = calculateRewardsTable(g)

#TODO: Instantiate your game.
#arbitrary start state
current_row_index = np.random.randint(m)
current_column_index = np.random.randint(m)
#continue choosing random row and column indexes until a non-terminal state is identified
#(i.e., until the chosen state is a 'white square').
while current_row_index == current_column_index:
  current_row_index = np.random.randint(m)
  current_column_index = np.random.randint(m)

start_state = (current_row_index,current_column_index)
game = CopsAndRobbers(g,start_state,Rewards)

here
true


In [404]:
# Simulation loop
tot_pen = 0
tot_steps = 0
#run for 100 iterations with the brute force method 
time_ave = 0
for iter in range(100):
  penalty = 0
  tot_steps = steps + tot_steps
  steps = 0
  playing = True
  t0 = time.process_time()
  while playing:
    #TODO: Get the next state, check for terminal state, and get reward
    next_state_cop = random.randint(0, g.m - 1)
    if next_state_cop in game.G[game.state[0]]: #is our next_state for the cop valid from our current one
      penalty = penalty + game.utility(game.state,next_state_cop) #add penalty from the rewards table
      game.result_cop(game.state,next_state_cop)
      playing = not game.terminal_test(game.state)
      if playing == False:
        continue
      game.result_rob(game.state)
      playing = not game.terminal_test(game.state) #this checks if robber moved 
      if playing == False:
        penalty == penalty + 100 # adds points because the robber landed on the cop 
        continue
      
    else:
      penalty = penalty + game.utility(game.state,next_state_cop)
      game.result_rob(game.state)
      playing = not game.terminal_test(game.state)
      if playing == False:
        penalty = penalty+100
    steps = steps+1
  t1 = time.process_time() - t0
  time_ave = t1 + time_ave 
  tot_pen = penalty + tot_pen
    # Make sure to break if you reach the terminal state.
tot_pen = tot_pen / 100
tot_steps = tot_steps / 100 
tot_time = t1 / 100
time_ave = time_ave / 100

print('Average Penalty Per Epoch: ' +str(tot_pen))
print('Average Steps Per Epoch: ' + str(tot_steps))
print('Average Time Per Epoch: ' + str(tot_time))

Average Penalty Per Epoch: 40.75
Average Steps Per Epoch: 8.88
Average Time Per Epoch: 8.071999999970103e-08


Q-Learn:

Average Reward Per Epoch: 48.52

Average Steps Per Epoch: 6.39

Average Time Per Epoch: 9.786122000093655e-05


Brute Force:

Average Penalty Per Epoch: 40.75

Average Steps Per Epoch: 8.88

Average Time Per Epoch: 8.071999999970103e-08

Using the brute force method with n = 5 and m = 10 we find that the reward is slightly lower than for the Q-learn when only iterating over 100 epochs. Around 8 steps are required per epoch which is more than that required for Q-learning. Finally, it should be noted that the time to run this approach is  faster, likely due to there being fewer function calls. Thus, for this few of epochs on a pretty uncomplex graph, the Q-learn doesn't significantly outperform the brute force approach.

Now let's test the brute force method against the Q-learning method with a larger graph and more epochs of training. Theoretically this is where Q-learning should really start to outperform brute. 

In [405]:
#Create a test graph to apply to both the brute force and other method

#Try for 25 total nodes and 10 ext. 
n = 10
m = 25
g = Graph(n,m)
g.make_random_graph()
#Run on Brute Force Method
Rewards = calculateRewardsTable(g)

#TODO: Instantiate your game.
#arbitrary start state
current_row_index = np.random.randint(m)
current_column_index = np.random.randint(m)
#continue choosing random row and column indexes until a non-terminal state is identified
#(i.e., until the chosen state is a 'white square').
while current_row_index == current_column_index:
  current_row_index = np.random.randint(m)
  current_column_index = np.random.randint(m)

start_state = (current_row_index,current_column_index)
game = CopsAndRobbers(g,start_state,Rewards)

here
true


In [408]:
# Simulation loop
tot_pen = 0
tot_steps = 0
#run for 100 iterations with the brute force method 
time_ave = 0
for iter in range(100):
  penalty = 0
  tot_steps = steps + tot_steps
  steps = 0
  playing = True
  t0 = time.process_time()
  while playing:
    #TODO: Get the next state, check for terminal state, and get reward
    next_state_cop = random.randint(0, g.m - 1)
    if next_state_cop in game.G[game.state[0]]: #is our next_state for the cop valid from our current one
      penalty = penalty + game.utility(game.state,next_state_cop) #add penalty from the rewards table
      game.result_cop(game.state,next_state_cop)
      playing = not game.terminal_test(game.state)
      if playing == False:
        continue
        print("playing")
      game.result_rob(game.state)
      playing = not game.terminal_test(game.state) #this checks if robber moved 
      if playing == False:
        penalty == penalty + 100 # adds points because the robber landed on the cop 
        continue
      
    else:
      penalty = penalty + game.utility(game.state,next_state_cop)
      game.result_rob(game.state)
      playing = not game.terminal_test(game.state)
      if playing == False:
        penalty = penalty+100
    steps = steps+1
  t1 = time.process_time() - t0
  time_ave = t1 + time_ave 
  tot_pen = penalty + tot_pen
    # Make sure to break if you reach the terminal state.
tot_pen = tot_pen / 100
tot_steps = tot_steps / 100 
tot_time = t1 / 100
time_ave = time_ave / 100

print('Average Penalty Per Epoch: ' +str(tot_pen))
print('Average Steps Per Epoch: ' + str(tot_steps))
print('Average Time Per Epoch: ' + str(tot_time))

Average Penalty Per Epoch: -5.02
Average Steps Per Epoch: 21.85
Average Time Per Epoch: 9.784999988937671e-08


Interestingly, when we increase the total nodes to 25, our penalty drops significantly (goes to around 0) and the total steps increase as well (around 20). This makes sense as we likely have many erroneous guesses during the brute force method. Time is not that different. Let's compare this to this larger graph for the q-learning method. See below.

In [410]:
#new rewards table
n = 10
m = 25

q_graph = Graph(n, m)
q_graph.make_random_graph()

#define the shape of the environment (i.e., its states)
environment_rows = m
environment_columns = m

#Create a 3D numpy array to hold the current Q-values for each state and action pair: Q(s, a) 
#The array contains 11 rows and 11 columns (to match the shape of the environment), as well as a third "action" dimension.
#The "action" dimension consists of 4 layers that will allow us to keep track of the Q-values for each possible action in
#each state (see next cell for a description of possible actions). 
#The value of each (state, action) pair is initialized to 0.
q_values = np.zeros((environment_rows, environment_columns, m))
#Create a 2D numpy array to hold the rewards for each state. 
#The array contains 11 rows and 11 columns (to match the shape of the environment), and each value is initialized to -1.
rewards = np.full((m,m, m), -5.)

for cop in range(m):
  for rob in range(m):
    for action in range(m):
      if (action in q_graph.G[cop]) & (action == rob):
        rewards[cop,rob,action] = 100
      elif (action in q_graph.G[cop]) & (action != rob):
        rewards[cop,rob,action] = -1
      #otherwise -5

here
true


In [412]:
epsilon = 0.9 #the percentage of time when we should take the best action (instead of a random action)
discount_factor = 0.9 #discount factor for future rewards
learning_rate = 0.9 #the rate at which the AI agent should learn

#run through 5000 training episodes
reward_tot = 0
reward_tracker =0
steps = 0
steps_tot = 0
time_ave = 0
t1 = 0
for episode in range(5000):
  #get the starting location for this episode
  row_index, column_index = get_starting_location()
  reward_tot = reward_tracker + reward_tot
  steps_tot = steps_tot + steps
  reward_tracker = 0 
  steps = 0
  time_ave = t1 + time_ave


  #continue taking actions (i.e., moving) until we reach a terminal state
  #(i.e., until we reach the item packaging area or crash into an item storage location)
  t0= time.process_time()
  while not is_terminal_state(row_index, column_index):
    #choose which action to take (i.e., where to move next)
    action_index = get_next_action(row_index, column_index, epsilon)

    #perform the chosen action, and transition to the next state (i.e., move to the next location)
    old_row_index, old_column_index = row_index, column_index #store the old row and column indexes
    row_index, column_index = get_next_location(old_row_index, old_column_index, action_index)
    
    #receive the reward for moving to the new state, and calculate the temporal difference
    reward = rewards[old_row_index, old_column_index,action_index]
    reward_tracker = reward_tracker + reward
    old_q_value = q_values[old_row_index, old_column_index, action_index]
    temporal_difference = reward + (discount_factor * np.max(q_values[row_index, column_index])) - old_q_value

    #update the Q-value for the previous state and action pair
    new_q_value = old_q_value + (learning_rate * temporal_difference)
    q_values[old_row_index, old_column_index, action_index] = new_q_value

    steps = steps+1
  t1 = time.process_time() - t0

#average reward across 5000 epochs
reward_tot = reward_tot/5000
steps_tot = steps_tot/5000
time_ave = time_ave/5000


print('Average Penalty Per Epoch: ' +str(reward_tot))
print('Average Steps Per Epoch: ' + str(steps_tot))
print('Average Time Per Epoch: ' + str(time_ave))


Average Penalty Per Epoch: 62.9026
Average Steps Per Epoch: 10.405
Average Time Per Epoch: 0.0002188608004000912


Indeed, with this more complex graph when we utilize Q-learning and increase the number of epochs we get much higher rewards (ie. lower penalty) and fewer steps than for the brute force approach. This also outperformed the brute force method even with fewer epochs (it outperformed even at like 500). Thus, it is clear that the reinforcement learning approach increases the effectiveness of the robber finding the cop.

> **Bonus** Check that the learned policy satsifies the [Bellman Inequality](https://towardsdatascience.com/mathematical-analysis-of-reinforcement-learning-bellman-equation-ac9f0954e19f), i.e is the computed solution the actual optimal policy?